In [196]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from copy import deepcopy
from sklearn.metrics import mean_squared_error

%matplotlib inline

# Data processing

### Read data

In [160]:
dta_fname = "../Data/Does_Price_Matter_AER_merged.dta"

raw_df = pd.read_stata(dta_fname)
colnames = raw_df.columns
print(colnames)

cols_to_keep = [
                'treatment', 'control',
                'ratio',
                # 'ratio2', 'ratio3',
                'size',
                # 'size25', 'size50', 'size100', 'sizeno',
                'ask',
                # 'ask1', 'ask2', 'ask3',
                # 'askd1', 'askd2', 'askd3',
                'amount', 'gave', 'amountchange'
               ]
df = raw_df[cols_to_keep]

df.head()

Index(['treatment', 'control', 'ratio', 'ratio2', 'ratio3', 'size', 'size25',
       'size50', 'size100', 'sizeno', 'ask', 'askd1', 'askd2', 'askd3', 'ask1',
       'ask2', 'ask3', 'amount', 'gave', 'amountchange', 'hpa', 'ltmedmra',
       'freq', 'years', 'year5', 'mrm2', 'dormant', 'female', 'couple',
       'state50one', 'nonlit', 'cases', 'statecnt', 'stateresponse',
       'stateresponset', 'stateresponsec', 'stateresponsetminc', 'perbush',
       'close25', 'red0', 'blue0', 'redcty', 'bluecty', 'pwhite', 'pblack',
       'page18_39', 'ave_hh_sz', 'median_hhincome', 'powner', 'psch_atlstba',
       'pop_propurban'],
      dtype='object')


,treatment,control,ratio,size,ask,amount,gave,amountchange
0,0,1,Control,Control,Control,0.0,0,-45.0
1,0,1,Control,Control,Control,0.0,0,-25.0
2,1,0,1,"$100,000",1x,0.0,0,-50.0
3,1,0,1,Unstated,1x,0.0,0,-25.0
4,1,0,1,"$50,000",1x,0.0,0,-15.0


### Remove control

In [169]:
# avg_control = np.mean(df[df["control"] == 1]["amountchange"])
# df_treatment = df[df["treatment"] == 1]
# df_treatment.loc[:, "amountchange"] = df_treatment.loc[:, "amountchange"] - avg_control
df_treatment = df

### Process data

In [170]:
ratio_map = {"Control": 0, 1: 1, 2: 2, 3: 3}
size_map = {"Control": 0, "$25,000": 1, "$50,000": 2, "$100,000": 3, "Unstated": 4}
ask_map = {"Control": 0, "1x": 1, "1.25x": 2, "1.50x": 3}

df_treatment.loc[:, "ratio"] = df_treatment.loc[:, "ratio"].map(ratio_map)
df_treatment.loc[:, "size"] = df_treatment.loc[:, "size"].map(size_map)
df_treatment.loc[:, "ask"] = df_treatment.loc[:, "ask"].map(ask_map)
df_treatment = df_treatment.astype({"ratio": np.int64, "size": np.int64, "ask": np.int64})

df_treatment = df_treatment.drop(["treatment", "control"], axis=1)

df_treatment.head()

,ratio,size,ask,amount,gave,amountchange
0,0,0,0,0.0,0,-45.0
1,0,0,0,0.0,0,-25.0
2,1,3,1,0.0,0,-50.0
3,1,4,1,0.0,0,-25.0
4,1,2,1,0.0,0,-15.0


In [288]:
Z = df_treatment.to_numpy()
X = Z[:, :3] #- 1
y = Z[:, 3] / 100 # (np.max(y) - np.min(y))
y = y.reshape((-1, 1))

num_data = X.shape[0]

# Analysis

### Rashomon

In [62]:
from rashomon import tva
from rashomon import loss
from rashomon import counter
from rashomon import metrics
from rashomon import extract_pools
from rashomon.aggregate import RAggregate_profile, RAggregate
from rashomon.sets import RashomonSet, RashomonProblemCache, RashomonSubproblemCache


%load_ext autoreload
%autoreload 2

Setup the policy and profile list

In [172]:
M = 3
# R = np.array([3, 4, 3])
R = np.array([4, 5, 4])

num_profiles = 2**M
profiles, profile_map = tva.enumerate_profiles(M)

all_policies = tva.enumerate_policies(M, R)
num_policies = len(all_policies)

In [181]:
policies_profiles = {}
policies_profiles_masked = {}
policies_ids_profiles = {}
# pi_policies = {}
# pi_pools = {}
for k, profile in enumerate(profiles):

    policies_temp = [(i, x) for i, x in enumerate(all_policies) if tva.policy_to_profile(x) == profile]
    unzipped_temp = list(zip(*policies_temp))
    policies_ids_k = list(unzipped_temp[0])
    policies_k = list(unzipped_temp[1])
    policies_profiles[k] = deepcopy(policies_k)
    policies_ids_profiles[k] = policies_ids_k

    
    profile_mask = list(map(bool, profile))

    # Mask the empty arms
    for idx, pol in enumerate(policies_k):
        policies_k[idx] = tuple([pol[i] for i in range(M) if profile_mask[i]])
    policies_profiles_masked[k] = policies_k

    # if np.sum(profile) > 0:
    #     pi_pools_k, pi_policies_k = extract_pools.extract_pools(policies_k, sigma[k])
    #     if len(pi_pools_k.keys()) != mu[k].shape[0]:
    #         print(f"Profile {k}. Expected {len(pi_pools_k.keys())} pools. Received {mu[k].shape[0]} means.")
    #     pi_policies[k] = pi_policies_k
    #     pi_pools[k] = {}
    #     for x, y in pi_pools_k.items():
    #         y_full = [policies_profiles[k][i] for i in y]
    #         y_agg = [all_policies.index(i) for i in y_full]
    #         pi_pools[k][x] = y_agg
    # else:
    #     pi_policies[k] = {0: 0}
    #     pi_pools[k] = {0: [0]}

In [252]:
D = np.zeros(shape=y.shape, dtype=np.int64)
profiles_in_data = []
for i in range(num_data):
    policy_i = tuple([int(x) for x in X[i, :]])
    policy_idx = [idx for idx in range(num_policies) if all_policies[idx] == policy_i]
    profiles_in_data.append(tva.policy_to_profile(policy_i))
    D[i, 0] = int(policy_idx[0])

policy_means = loss.compute_policy_means(D, y, num_policies)

In [380]:
mu_policies = policy_means[:, 0] / policy_means[:, 1]
mu_policies[np.isnan(mu_policies)] = -np.inf

print(mu_policies)
print(np.max(mu_policies))
all_policies[np.argmax(mu_policies)]

[0.00081327       -inf       -inf       -inf       -inf       -inf
       -inf       -inf       -inf       -inf       -inf       -inf
       -inf       -inf       -inf       -inf       -inf       -inf
       -inf       -inf       -inf       -inf       -inf       -inf
       -inf 0.00061207 0.00071367 0.00092233       -inf 0.0008127
 0.0009806  0.00089189       -inf 0.00078041 0.0015162  0.00078664
       -inf 0.00092573 0.00127077 0.00102909       -inf       -inf
       -inf       -inf       -inf 0.00142179 0.0008127  0.001507
       -inf 0.00057514 0.00133621 0.00102047       -inf 0.00052859
 0.00086207 0.0011153        -inf 0.00110118 0.00122306 0.00080819
       -inf       -inf       -inf       -inf       -inf 0.0012438
 0.00090517 0.00140281       -inf 0.00081122 0.00088134 0.00069073
       -inf 0.0012605  0.00057929 0.00070259       -inf 0.0008903
 0.00096444 0.00092171]
0.001516198704103672


/var/folders/ds/v36htzx10pv39ydvb7plxs1r0000gn/T/ipykernel_25288/2539375452.py:1: RuntimeWarning: invalid value encountered in divide
  mu_policies = policy_means[:, 0] / policy_means[:, 1]


(1, 3, 2)

In [386]:
np.argsort(mu_policies)

array([76, 24, 28, 60, 32, 36, 72, 40, 41, 23, 42, 44, 48, 68, 52, 64, 56,
       63, 62, 43, 22, 61, 20,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 21,
       12, 13, 11, 19, 18, 17, 15, 14, 16, 53, 49, 74, 25, 71, 75, 26, 33,
       35, 59, 69, 46, 29,  0, 54, 70, 77, 31, 66, 79, 27, 37, 78, 30, 51,
       39, 57, 55, 58, 65, 73, 38, 50, 67, 45, 47, 34])

In [376]:
D[np.argmax(policy_means)]

array([0])

In [373]:
R_set, rashomon_profiles = RAggregate(M, R, 24, D, y, 0.007583, reg=1e-8)

print(len(R_set))

13


In [381]:
best_profile_counter = np.zeros(shape=(num_profiles,))

all_losses = []

reg = 1e-6
best_loss = np.inf
best_policy_o = None

for r_set in R_set:
    pi_policies_profiles_r = {}
    
    for k, profile in enumerate(profiles):
        if rashomon_profiles[k].sigma[r_set[k]] is None:
            pi_policies_r_k = {i: 0 for i in range(len(policies_profiles_masked[k]))}
        else:
            _, pi_policies_r_k = extract_pools.extract_pools(
                policies_profiles_masked[k],
                rashomon_profiles[k].sigma[r_set[k]]
            )
            # print(pi_policies_r_k)
        pi_policies_profiles_r[k] = pi_policies_r_k
    
    pi_pools_r, pi_policies_r = extract_pools.aggregate_pools(pi_policies_profiles_r, policies_ids_profiles)
    pool_means_r = loss.compute_pool_means(policy_means, pi_pools_r)
    y_pred = metrics.make_predictions(D, pi_policies_r, pool_means_r)

    this_loss = mean_squared_error(y, y_pred) + reg * len(pi_pools_r)
    all_losses.append((this_loss, mean_squared_error(y, y_pred), len(pi_pools_r)))

    # print(pi_pools_r)
    
    # best_pools = np.argwhere(pool_means_r == np.max(pool_means_r))[0]
    best_pools = np.argwhere(pool_means_r == np.nanmax(pool_means_r))[0]

    for best_pool in best_pools:
        best_policy_id = pi_pools_r[best_pool][0]
        best_policy = all_policies[best_policy_id]
        best_profile = tva.policy_to_profile(best_policy)
        profile_id = profile_map[best_profile]

        best_profile_counter[profile_id] += 1
        
        if this_loss < best_loss:
            best_loss = this_loss
            best_policy_o = best_policy
            print(len(pi_pools_r))
            for k, v in pi_pools_r.items():
                print(k, ":", [all_policies[x] for x in v], v)
                print(pool_means_r[k])

best_profile_freq = best_profile_counter / np.sum(best_profile_counter)

print(f"Best model loss {best_loss} and best policy {best_policy_o}")

19
0 : [(0, 0, 0)] [0]
0.0008132678132678132
1 : [(0, 0, 1), (0, 0, 2), (0, 0, 3)] [1, 2, 3]
nan
2 : [(0, 1, 0), (0, 2, 0), (0, 3, 0), (0, 4, 0)] [4, 8, 12, 16]
nan
3 : [(0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 2, 1), (0, 2, 2), (0, 2, 3), (0, 3, 1), (0, 3, 2), (0, 3, 3), (0, 4, 1), (0, 4, 2), (0, 4, 3)] [5, 6, 7, 9, 10, 11, 13, 14, 15, 17, 18, 19]
nan
4 : [(1, 0, 0), (2, 0, 0), (3, 0, 0)] [20, 40, 60]
nan
5 : [(1, 0, 1), (1, 0, 2), (1, 0, 3), (2, 0, 1), (2, 0, 2), (2, 0, 3), (3, 0, 1), (3, 0, 2), (3, 0, 3)] [21, 22, 23, 41, 42, 43, 61, 62, 63]
nan
6 : [(1, 1, 0), (1, 2, 0), (1, 3, 0), (1, 4, 0), (2, 1, 0), (2, 2, 0), (2, 3, 0), (2, 4, 0), (3, 1, 0), (3, 2, 0), (3, 3, 0), (3, 4, 0)] [24, 28, 32, 36, 44, 48, 52, 56, 64, 68, 72, 76]
nan
7 : [(1, 1, 1)] [25]
0.0006120689655172415
8 : [(1, 1, 2)] [26]
0.0007136706135629711
9 : [(1, 1, 3)] [27]
0.0009223300970873788
10 : [(1, 2, 1), (1, 3, 1), (1, 4, 1)] [29, 33, 37]
0.0008396124865446718
11 : [(1, 2, 2), (1, 3, 2), (1, 4, 2)] [30, 34, 38]
0.00